In [1]:
import numpy as np
import pandas as pd
import random
import matplotlib
import matplotlib.pyplot as plt
from datetime import date
import sys
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
from sklearn.cluster import KMeans
matplotlib.rc('xtick', labelsize=15) 
from yellowbrick.cluster import KElbowVisualizer
from sklearn.preprocessing import StandardScaler
import json
import requests
from google.cloud import storage
from io import StringIO
from read_in_data_v2 import *
from jtech_fxns_v2 import *
from tagging_fxns_v2 import * 
from ab_split_fxns_v2 import *

In [2]:
def get_daily_strings():
    with open('keys_v2.json', 'r') as file:
        config = json.load(file)
        return config["jwt"], config["bucket_name"]

In [3]:
def read_in_all_csvs(table_name, bucket):
    data = bucket.blob(table_name).download_as_bytes()
    string_data = StringIO(data.decode('utf-8'))
    output_df = pd.read_csv(string_data, on_bad_lines='warn')
    print(f'{table_name} ingested to pandas dataframe...')
    return output_df

In [140]:
def clean_cols(df, package_name, platform):
    #remove blank looker columns w/ id and name columns consistently
    cleaned_cols = [str.lower(str.replace(col_name, " ", "_")) for col_name in df.columns]
    df.columns = cleaned_cols
    df = df[(df.package_name == package_name) & (df.platform == platform)].drop(columns=['package_name', 'platform'])
    if 'unnamed:_0' in df.columns:
        df = df.drop(columns=['unnamed:_0'])
    return df


def add_iap_cols(iap):
    iap['avg_trxn_amt_7d'] = iap.iap_usd_last_7d / iap.iap_trxns_last_7d
    iap['avg_cs_trxn_amt_7d'] = iap.cs_iap_last_7d / iap.cs_iap_trxns_last_7d
    iap['avg_trxn_amt_30d'] = iap.iap_usd_last_30d / iap.iap_trxns_last_30d
    iap['avg_cs_trxn_amt_30d'] = iap.cs_iap_last_30d / iap.cs_iap_trxns_last_30d
    iap['sent_to_click'] =( iap.cs_clicks / iap.cs_sent).round(3)
    iap['cs_click_freq_10d'] = (iap.cs_clicks_10d / iap.cs_sent_10d).round(3)
    iap.replace([np.inf, -np.inf], 0, inplace=True)
    return iap

In [5]:
def assign_clusters_kmeans(segment_sub, array_cols, k_clusters, label):
    segment_labels = np.array(segment_sub['user_id'])
    array = np.array(segment_sub[array_cols])
    km = KMeans(k_clusters)
    scaler = StandardScaler()
    scaled_array = scaler.fit_transform(array)
    y=km.fit_predict(scaled_array)
    cluster_labels = pd.DataFrame({'user_id': segment_labels, f'segment_{label}': y}, columns=['user_id', f'segment_{label}'])
    cluster_labels[f'segment_{label}'] = cluster_labels[f'segment_{label}'].astype(str)
    return array, cluster_labels


In [6]:
def get_segments_all_users(iap, cluster_labels_cspayers):
    scaler = StandardScaler()
    iap = iap.fillna(0).copy()
    non_cs_cols = ['user_id', 'iap_all', 'max_trxn_amt', 'iap_trxns_last_30d', 'iap_usd_last_30d']

    non_cs_attributes_payer = iap[iap.cs_iap > 0][non_cs_cols].copy().set_index('user_id')
    payer_labels = non_cs_attributes_payer.index
    payer_array = non_cs_attributes_payer.to_numpy()
    payer_array = scaler.fit_transform(payer_array)
    
    non_cs_attributes_nonpayer = iap[iap.cs_iap == 0][non_cs_cols].set_index('user_id')
    nonpayer_labels = non_cs_attributes_nonpayer.index
    nonpayer_array = non_cs_attributes_nonpayer.to_numpy()
    nonpayer_array = scaler.transform(nonpayer_array)
    
    most_similar_idxs = get_closest_dist_idxs(payer_array, nonpayer_array)
    most_sim_payers =pd.DataFrame(payer_labels[most_similar_idxs])
    most_sim_payers['segment'] = most_sim_payers['user_id'].map(cluster_labels_cspayers.set_index('user_id')['segment_cspayers'])

    most_sim_segments = most_sim_payers['segment'].to_numpy()
    cluster_labels_noncspayers = pd.DataFrame(index=nonpayer_labels, data = most_sim_segments, columns = ['segment_cspayers']).reset_index()

    users_clustered = pd.concat([cluster_labels_cspayers, cluster_labels_noncspayers], axis = 0, ignore_index=True)
    users_clustered.columns = ['user_id', 'segment']
    return users_clustered

In [7]:
def get_closest_dist_idxs(array1, array2):

    distance_matrix = np.linalg.norm(array2[:, np.newaxis] - array1, axis=2)
    most_similar_idxs = np.argmin(distance_matrix, axis=1)
    return most_similar_idxs

In [8]:
def jtech_transform_dfs_sim(pb, iap, pm, array_cols):
    
    #map the offer to the price
    pb['price'] = pb['offer'].map(pm.set_index('offer')['price']) 
    #remove n/as for price
    pb = pb[pd.notna(pb.price)].copy()
    #group by msg/click/iap event for each offer id (note: we're treating all cadence groups the same here)
    pb_grouped = pb[(pb.event_name == 'sent')| (pb.event_name == 'iap')|(pb.event_name == 'click')].groupby(by=['user_id', 'offer', 'price', 'event_name']).sum().reset_index() 

    #cluster users who have purchased CS
    iap.cs_iap = iap.cs_iap.astype(float)
    segment_sub_payers = iap[iap.cs_iap >0].copy().fillna(0)
    array_cspayers, cluster_labels_cspayers = assign_clusters_kmeans(segment_sub_payers, array_cols, k_clusters, 'cspayers')
    
    #for non-CS payers find the most similar CS payers to determine segment
    all_users_clustered = get_segments_all_users(iap, cluster_labels_cspayers)
    
    #merge CS payer clusters with the original pb_grouped table to get a count for sent, click, and iap for each offer and each "segment"
    cs_payers_clusters_merged = pd.merge(cluster_labels_cspayers, pb_grouped, how = 'left', left_on='user_id'
                , right_on = 'user_id')[['user_id', 'offer', 'price','event_name', 'cadence_stats', 'segment_cspayers']].reset_index().copy().drop(columns=['index']).dropna()

    cs_payers_clusters_grouped = cs_payers_clusters_merged.groupby(by=['segment_cspayers', 'event_name', 'offer', 'price']).sum().reset_index()[['segment_cspayers', 'event_name', 'offer', 'price','cadence_stats']]

    return pb_grouped, array_cspayers, cluster_labels_cspayers, cs_payers_clusters_grouped, cs_payers_clusters_merged, all_users_clustered


In [9]:
def jtech_create_arrays_cspayers(cs_payers_clusters_grouped):
    
    event_names = np.sort(np.unique(cs_payers_clusters_grouped['event_name']))  #save event names (example: "click")
    segments = np.sort(np.unique(cs_payers_clusters_grouped['segment_cspayers']))       # save cluster names
    iix = pd.MultiIndex.from_product([event_names, segments]) #create multi index
    
    
    cs_payers_clusters_pivot = cs_payers_clusters_grouped.pivot_table('cadence_stats', ['event_name', 'segment_cspayers'], 'offer', aggfunc='first').reindex(iix).fillna(0)

    offers = cs_payers_clusters_pivot.columns
    prices = pm.set_index('offer').loc[offers, 'price'].values
    
    #array shape: (events, num_clusters, num_offers)
    array = cs_payers_clusters_pivot.to_numpy().reshape(len(event_names),len(segments),-1)
    
    return event_names, segments, offers, prices, array

In [10]:
def jtech_get_scores(click_wt, msgs_wt, iap_wt, array, offers):
    clicks = array[0] * click_wt
    msgs_sent = array[2] * msgs_wt
    iap_success = array[1] * iap_wt
    output = clicks+msgs_sent+iap_success
    return output #these are the scores for each cluster (shape is cluster, offer, and values are the score)

In [70]:
def jtech_assign_offers_df(scores, users_clustered, offers, jtech_segments, pm):
    #method to assign a "weight" for each score (subject to change). right now it's just delta from min score across offers within each cluster
    delta_from_min = scores - np.min(scores, axis=1).reshape(-1,1)
    delta_from_min_sum_reshaped = delta_from_min.sum(axis=1)[:, np.newaxis]
    weights_normalized = delta_from_min / delta_from_min_sum_reshaped #these are the weighted probabilities for each offer within each cluster
    
    df = users_clustered[['user_id', 'segment']].copy()
    df['offer_to_send'] = df['segment'].apply(lambda x: jtech_choose_offer_weighted_prob(x, offers, weights_normalized, jtech_segments, scores)) #for each user, randomly assign offer based on weighted probability for that price within that cluster
    df['price'] = df['offer_to_send'].map(pm.set_index('offer')['price']) 
    df_agg_offer = df.groupby(by=['segment', 'offer_to_send']).count().reset_index() 
    df_agg_price = df.groupby(by=['segment', 'price']).count().reset_index() 

    return df, df_agg_offer, df_agg_price

In [12]:
def jtech_choose_offer_weighted_prob(segment, offers, weights_normalized, jtech_segments, scores):
    # this is the row-level function for determining offer based on weighted probability for that price within the cluster/segment
    try:
        idx = np.where(jtech_segments == segment)[0][0]
    except:
        idx = 0
    return offers[np.where(scores[idx,:] == np.random.choice(scores[idx, :], p=weights_normalized[idx, :]))][0]

In [78]:
#elbow plot for viz
def elbow_plot(array):
    #run this to figure out best # of clusters
    model = KMeans()
    scaler = StandardScaler()
    scaled_array = scaler.fit_transform(array)
    visualizer = KElbowVisualizer(model, k=(2,14), timings= True)
    visualizer.fit(scaled_array)        # Fit data to visualizer
    visualizer.show()        # Finalize and render figure

In [79]:
def jtech(pb, iap, pm, k_clusters, array_cols, click_wt=-0.2, msgs_wt=-0.1, iap_exp=2):

    #jtech: get the clusters for jtech
    pb_grouped, array_cspayers, cluster_labels_cspayers, cs_payers_clusters_grouped, cs_payers_clusters_merged, all_users_clustered = jtech_transform_dfs_sim(pb, iap, pm, array_cols)

    #jtech: create arrays for getting jtech scores of each cluster/segment. then get scores for each cluster
    event_names, jtech_segments, offers, prices, jtech_array = jtech_create_arrays_cspayers(cs_payers_clusters_grouped)
    scores = jtech_get_scores(click_wt, msgs_wt, iap_wt = (np.array(prices)**iap_exp), array=jtech_array, offers=offers)

    # jtech: use scores and weighted probability of the score to assign an offer for each user
    df_jtech, df_agg_offers, df_agg_price= jtech_assign_offers_df(scores, all_users_clustered, offers, jtech_segments, pm)

    df_jtech = pd.merge(df_jtech, iap[['user_id', 'cs_click_freq_10d', 'cs_iap_last_7d', 'sent_to_click', 'cs_sent']], on='user_id', how='left').fillna(0)

    return df_jtech, df_agg_jtech, jtech_segments, offers, scores, jtech_array, array_cspayers

In [13]:
today = str.replace(str(date.today()),"-","")
jwt, bucket_name = get_daily_strings()
storage_client = storage.Client()
bucket = storage_client.get_bucket(bucket_name)
# predefined_cohorts = sys.argv[1] #very first time you run a new ab test (Regardless of cadence group) this should be 'no' otherwise it should be 'yes'
# cadence_type = sys.argv[2] 

/Users/jamie.wessels/anaconda3/lib/python3.11/site-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)
/Users/jamie.wessels/anaconda3/lib/python3.11/site-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [148]:
package_names = ['com.loop.match3d']
platforms = ['ios']
k_clusters_list = [6]
cadence_group = 'session15'

#jtech variables:
iap_exp = 2
segment = 'segment' 
array_cols = ['iap_all', 'max_trxn_amt','iap_trxns_last_30d', 'max_cs_trxn_amt','cs_clicks', 'cs_iap_last_30d'] #for kmeans
excluded_offers = ['match3d.candlestick.099', 'match3d.candlestick.29999']

In [149]:
iap_all, pb_all, pl_all, pm_all = (read_in_all_csvs(table, bucket) for table in ['iap_stats_v6', 'past_behavior_v2', 'price_lookups_v2', 'price_mapping'])


iap_stats_v6 ingested to pandas dataframe...
past_behavior_v2 ingested to pandas dataframe...
price_lookups_v2 ingested to pandas dataframe...
price_mapping ingested to pandas dataframe...


In [156]:
for package_name, platform, k_clusters in zip(package_names, platforms, k_clusters_list):
        print(f'Starting {package_name} {platform}...')
        iap, pb, pl, pm = (clean_cols(df, package_name, platform) for df in [iap_all, pb_all, pl_all, pm_all])
        # users_to_exclude_nonengagement, users_to_exclude_testers = get_users_to_exclude(pb, matches, iap)
        iap = add_iap_cols(iap).copy()
        pb = pb[~pb.offer.isin(excluded_offers)].copy()

Starting com.loop.match3d ios...


In [157]:
df_jtech, df_agg_jtech, jtech_segments, offers, scores, jtech_array, array_cspayers = jtech(pb, iap, pm, k_clusters, array_cols, click_wt=-0.2, msgs_wt=-0.1, iap_exp=2)


In [158]:
df_jtech.groupby('price').count()['user_id'].reset_index().sort_values('price')

,price,user_id
0,2.99,30
1,4.99,66
2,6.99,50
3,9.99,177
4,12.99,707
5,24.99,5741
6,34.99,1999
7,49.99,661
8,99.99,184
9,199.99,16


In [159]:
df_jtech.groupby('segment').count()['user_id']

segment
0      14
1    8962
2      39
3     182
4       4
5     430
Name: user_id, dtype: int64

In [86]:
test = pb.groupby(['user_id', 'cadence_group', 'event_name']).sum()['cadence_stats'].reset_index()

In [160]:
def cvr_by_cadence_group(pb, cadence_group):
    df = pb.groupby(['user_id', 'cadence_group', 'event_name']).sum()['cadence_stats'].reset_index()
    df_pivot = df.pivot_table('cadence_stats', ['user_id', 'cadence_group'], 'event_name').fillna(0).reset_index()
    df_pivot['ctr'] = (df_pivot['click'] / df_pivot['sent']).fillna(0)
    df_pivot.replace([np.inf, -np.inf], 0, inplace=True)
    
    ctr_output = df_pivot.pivot_table('ctr', ['user_id'], 'cadence_group').fillna(0).reset_index()[['user_id', cadence_group]]
    iap_output = df_pivot.pivot_table('iap', ['user_id'], 'cadence_group').fillna(0).reset_index()[['user_id', cadence_group]]

    return ctr_output, iap_output

In [166]:
ctr_session, iap_session = cvr_by_cadence_group(pb, 'pbo')

In [178]:
ctr_session.columns

Index(['user_id', 'pbo'], dtype='object', name='cadence_group')

In [176]:
ctr_session.head()

cadence_group,user_id,pbo
0,ua0000f6f99a67db092e86d0aa04a33bc7,0.0
1,ua00069cf817acb887704c6e2864837505,0.0
2,ua0015bdf2a034e6548eb511e2c226e245,0.0
3,ua0017953adc3161e29e8adb547c503717,0.0
4,ua0018685439bac0decd40896ce0f7c1f4,0.0


In [167]:
ctr_session.describe()

cadence_group,pbo
count,12000.000000
mean,0.015505
std,0.075755
min,0.000000
25%,0.000000
50%,0.000000
75%,0.000000
max,1.750000


In [168]:
iap_session.describe()

cadence_group,pbo
count,12000.000000
mean,0.008917
std,0.194954
min,0.000000
25%,0.000000
50%,0.000000
75%,0.000000
max,12.000000


In [173]:
ctr_session.sort_values('pbo', ascending=False).head(30)

cadence_group,user_id,pbo
4857,ua66a8695da7cf3d828a49722b453a58be,1.750000
11196,uaee72ee5d12e9e253ec8807c2dee586d4,1.250000
8544,uab68daa7f0dcc297ebcb6bbc04fc86beb,1.250000
10460,uadf1f5d9be2cf91c8acc166f257ed2e30,1.142857
3222,ua43e67fb934ac367ff9f7d39d2fbc7c27,1.000000
6281,ua859ce2d2ca2c4547d68b219aead8aa08,1.000000
6424,ua888a13224c475d9fa70d097ae4f625e4,1.000000
1203,ua19355405d7ec241a08847ba7ddff7cc6,0.857143
8293,uab0d653d9d729aa2b9fe15f04cc9324c1,0.833333
9098,uac211650959bc98802789dcea2ada027c,0.833333


In [175]:
iap[iap.user_id=='ua462bf89f2222605ec649a6b9e5f9bc18']

,user_id,iap_all,cs_iap,max_trxn_amt,iap_trxns_last_30d,iap_trxns_last_7d,iap_usd_last_30d,max_cs_trxn_amt,iap_usd_last_7d,cs_clicks,...,cs_iap_trxns_10d,cs_clicks_10d,cs_iap_trxns_last_7d,cs_iap_trxns_last_30d,avg_trxn_amt_7d,avg_cs_trxn_amt_7d,avg_trxn_amt_30d,avg_cs_trxn_amt_30d,sent_to_click,cs_click_freq_10d
64,ua462bf89f2222605ec649a6b9e5f9bc18,3160.32,449.97,199.99,21.0,4.0,566.79,199.99,120.96,40.0,...,0.0,1.0,0.0,0.0,30.24,NaN,26.99,NaN,0.656,0.333
